# Orchestration Service Tutorial

This notebook demonstrates how to use the SDK to interact with the Orchestration Service, enabling the creation of AI-driven workflows by seamlessly integrating various modules, such as templating, large language models (LLMs), data masking and content filtering. By leveraging these modules, you can build complex, automated workflows that enhance the capabilities of your AI solutions. For more details on configuring and using these modules, please refer to the [Orchestration Service Documentation](https://help.sap.com/docs/ai-launchpad/sap-ai-launchpad/orchestration).

## NOTE:
### Please download the latest version of GenAi Hub v3.2.0(scheduled release date 7th Oct and beyond) from https://pypi.org/project/generative-ai-hub-sdk/ to execute this notebook

To check the installed version of generative-ai-hub-sdk:

In [1]:
!pip show generative-ai-hub-sdk

Name: generative-ai-hub-sdk
Version: 3.1.1
Summary: generative AI hub SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages
Requires: ai-core-sdk, click, dacite, langchain, langchain-community, openai, overloading, pydantic
Required-by: 


Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 10.156 seconds


## Initializing the Orchestration Service

⚠️Before using the SDK, you need to set up a virtual deployment of the Orchestration Service. Once deployed, you'll have access to a unique endpoint URL (deploymentUrl).

In [2]:
YOUR_API_URL = "..."

## Basic Orchestration Pipeline

Now that you have YOUR_API_URL, let's walk through a basic orchestration pipeline for a translation task.

### Step 1: Define the Template and Default Input Values

The Template class is used to define structured message templates for generating dynamic interactions with language models. In this example, the template is designed for a translation assistant, allowing users to specify a language and text for translation.

In [3]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?text}}"
        ),
    ],
    defaults=[
        TemplateValue(name="to_lang", value="German"),
    ],
)

Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 3.422 seconds


This template can be used to create translation requests where the language and text to be translated are specified dynamically. The placeholders in the UserMessage will be replaced with the actual values provided at runtime, and the default value for the language is set to German.

### Step 2: Define the LLM

The LLM class is used to configure and initialize a language model for generating text based on specific parameters. In this example, we'll use the GPT-4o model to perform the translation task.

ℹ️Note that virtual deployment of the language model is managed automatically by the Orchestration Service, so no additional deployment setup is required on your part.

In [4]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 256, "temperature": 0.2})

This configuration initializes the language model to use the gpt-4o variant with the latest updates. The model will generate responses up to 256 tokens in length and produce more predictable and focused output due to the low temperature setting.

### Step 3: Create the Orchestration Configuration

The OrchestrationConfig class is used to create a configuration that integrates various components, such as templates and language models, into a unified orchestration setup. This configuration specifies how these components work together to achieve the desired workflow.

In [5]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

### Step 4: Run the Orchestration Request

The OrchestrationService class is used to interact with the orchestration service by providing a configuration and invoking its operations. This service handles the execution of workflows defined by the provided configuration and processes inputs accordingly.

In [6]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)

Call the run method with the required template_values. The service will process the input according to the configuration and return the result.

In [7]:
result = orchestration_service.run(template_values=[
    TemplateValue(name="text", value="The Orchestration Service is working!")
])
print(result.orchestration_result.choices[0].message.content)

Der Orchestrierungsdienst funktioniert!


Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 4.625 seconds


## Optional Modules

### Data Masking

The Data Masking Module anonymizes or pseudonymizes personally identifiable information (PII) before it is processed by the LLM module. When data is anonymized, all identifying information is replaced with placeholders (e.g., MASKED_ENTITY), and the original data cannot be recovered, ensuring that no trace of the original information is retained. In contrast, pseudonymized data is substituted with unique placeholders (e.g., MASKED_ENTITY_ID), allowing the original information to be restored if needed. In both cases, the masking module identifies sensitive data and replaces it with appropriate placeholders before further processing.

In [8]:
from gen_ai_hub.orchestration.utils import load_text_file
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, \
    ProfileEntity

data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION
            ]
        )
    ]
)

config = OrchestrationConfig(
    template=Template(
        messages=[
            SystemMessage("You are a helpful AI assistant."),
            UserMessage("Summarize the following CV in 10 sentences: {{?orgCV}}"),
        ]
    ),
    llm=LLM(
        name="gpt-4o",
    ),
    data_masking=data_masking
)

cv_as_string = load_text_file("data/cv.txt")

result = orchestration_service.run(
    config=config,
    template_values=[
        TemplateValue(name="orgCV", value=cv_as_string)
    ]
)

In [9]:
print(result.orchestration_result.choices[0].message.content)

MASKED_PERSON is a seasoned financial expert with a strong focus on strategic and financial planning, accurate forecasting, and process implementation. They hold a Master of Science in Finance (2014) and a Bachelor of Science in Finance (2011) from MASKED_ORG. Their skill set includes proficiency in SAP and Excel VBA, as well as staff leadership and development. They are a Certified Management Accountant. In their role as a Financial Manager from 2016 to 2018 at MASKED_ORG, they managed all financial processes and devised short and long-term financial strategies to meet company goals. They also recommended innovative ways to generate revenue and cut costs while conducting advanced deal analysis and negotiations with potential investors. Their earlier experience from 2013 to 2016 involved drafting executive reports that outlined business issues, risks, and profit opportunities. Throughout both roles, they employed market research and surveys to drive business performance improvements. O

Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 5.688 seconds


### Content Filtering

The Content Filtering Module can be configured to filter both the input to the LLM module (input filter) and the output generated by the LLM (output filter). The module uses predefined classification services to detect inappropriate or unwanted content, allowing flexible configuration through customizable thresholds. These thresholds can be set to control the sensitivity of filtering, ensuring that content meets desired standards before it is processed or returned as output.

In [10]:
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter, AzureThreshold

input_filter= AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                  violence=AzureThreshold.ALLOW_SAFE,
                                  self_harm=AzureThreshold.ALLOW_SAFE,
                                  sexual=AzureThreshold.ALLOW_SAFE)
output_filter = AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                   violence=AzureThreshold.ALLOW_SAFE_LOW,
                                   self_harm=AzureThreshold.ALLOW_SAFE_LOW_MEDIUM,
                                   sexual=AzureThreshold.ALLOW_ALL)

config = OrchestrationConfig(
    template=Template(
        messages=[
            SystemMessage("You are a helpful AI assistant."),
            UserMessage("{{?text}}"),
        ]
    ),
    llm=LLM(
        name="gpt-4o",
    ),
    input_filters=[input_filter],
    output_filters=[output_filter]
)

In [11]:
from gen_ai_hub.orchestration.exceptions import OrchestrationError

try:
    result = orchestration_service.run(config=config, template_values=[
        TemplateValue(name="text", value="I hate you")
    ])
except OrchestrationError as error:
    print(error.message)

Content filtered due to Safety violations. Please modify the prompt and try again.


## Advanced Examples

In [12]:
service = OrchestrationService(api_url=YOUR_API_URL)

### Translation Service

In [13]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class TranslationService:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful translation assistant."),
                    UserMessage(
                        "Translate the following text to {{?to_lang}}: {{?text}}"
                    ),
                ],
                defaults=[
                    TemplateValue(name="to_lang", value="English"),
                ],
            ),
            llm=LLM(name="gpt-4o"),
        )

    def translate(self, text, to_lang):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="to_lang", value=to_lang),
                TemplateValue(name="text", value=text),
            ],
        )

        return response.orchestration_result.choices[0].message.content


In [14]:
translator = TranslationService(orchestration_service=service)

In [15]:
result = translator.translate(text="Hello, world!", to_lang="French")
print(result)

Bonjour, le monde !


Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 4.531 seconds


In [16]:
result = translator.translate(text="Hello, world!", to_lang="Spanish")
print(result)

¡Hola, mundo!


In [17]:
result = translator.translate(text="Hello, world!", to_lang="German")
print(result)

Hallo, Welt!


### Chatbot with Memory

In [18]:
from typing import List

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            llm=LLM(name="gpt-4"),
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

In [19]:
bot = ChatBot(orchestration_service=service)

In [20]:
print(bot.chat("Hello, how are you?"))

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [21]:
print(bot.chat("What's the weather like today?"))

I don't have real-time capabilities to check current weather conditions. However, you can easily find the weather by checking a weather website like Weather.com, using a weather app on your smartphone, or by asking a voice-activated device like Google Home or Amazon Alexa. How else may I assist you today?


In [22]:
print(bot.chat("Can you remember what I first asked you?"))

Yes, you first asked, "Hello, how are you?" How can I assist you further?


### Sentiment Analysis with Few Shot Learning 

In [23]:
from typing import List, Tuple

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import (
    SystemMessage,
    UserMessage,
    AssistantMessage,
)
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class FewShotLearner:
    def __init__(
            self,
            orchestration_service: OrchestrationService,
            system_message: SystemMessage,
            examples: List[Tuple[UserMessage, AssistantMessage]],
    ):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=self._create_few_shot_template(system_message, examples),
            llm=LLM(name="gpt-35-turbo"),
        )

    @staticmethod
    def _create_few_shot_template(
            system_message: SystemMessage,
            examples: List[Tuple[UserMessage, AssistantMessage]],
    ) -> Template:
        messages = [system_message]

        for example in examples:
            messages.append(example[0])
            messages.append(example[1])
        messages.append(UserMessage("{{?user_input}}"))

        return Template(messages=messages)

    def predict(self, user_input: str) -> str:
        response = self.service.run(
            config=self.config,
            template_values=[TemplateValue(name="user_input", value=user_input)],
        )

        return response.orchestration_result.choices[0].message.content

In [24]:
sentiment_examples = [
    (UserMessage("I love this product!"), AssistantMessage("Positive")),
    (UserMessage("This is terrible service."), AssistantMessage("Negative")),
    (UserMessage("The weather is okay today."), AssistantMessage("Neutral")),
]

In [25]:
sentiment_analyzer = FewShotLearner(
    orchestration_service=service,
    system_message=SystemMessage(
        "You are a sentiment analysis assistant. Classify the sentiment as Positive, Negative, or Neutral."
    ),
    examples=sentiment_examples,
)

In [26]:
print(sentiment_analyzer.predict("The movie was a complete waste of time!"))

Negative


In [27]:
print(
    sentiment_analyzer.predict("The traffic was fortunately unusually light today.")
)

Positive


In [28]:
print(
    sentiment_analyzer.predict("I'm not sure how I feel about the recent events.")
)

Neutral


Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 21.547 seconds
